## Code for analyzing metagenomic sequencing data

Code run locally to analyze metagenomic sequences from mice inoculated with primate gut microbiomes. Names of scripts run on an HPC are also noted.

### Trimming and quality filtering

These were performed using the kneaddata_array.sh script.

### Microbiome profiling

This was performed using the humann2_array.sh script. Normalization and regrouping were performed using the humann2_renorm.sh and humann2_regroup_renorm.sh scripts.

### Split tables

Humann2 output was split into stratified and unstratified tables before downstream analysis.

In [ ]:
humann2_split_stratified_table --input mouse_genefamilies_cpm.tsv --output mouse_genefamilies_cpm

humann2_split_stratified_table --input mouse_pathabundance_relab.tsv --output mouse_pathabundance_relab

humann2_split_stratified_table --input mouse_genefamilies_ko.tsv --output mouse_genefamilies_ko

humann2_split_stratified_table --input mouse_pathabundance.tsv --output mouse_pathabundance


### Beta diversity calculations

QIIME2 (v2019.10) was used to construct PCoA plots and calculate beta-diversity metrics. Only samples from adult-innoculated mice were analyzed.

In [ ]:
source activate qiime2-2019.10

biom convert -i /mouse_pathabundance/mouse_pathabundance_unstratified.tsv \
    -o /mouse_pathabundance/mouse_pathabundance_unstratified.biom --table-type "Pathway table" --to-hdf5

biom convert -i /mouse_genefamilies_ko/mouse_genefamilies_ko_unstratified.tsv \
    -o /mouse_genefamilies_ko_unstratified.biom --table-type="OTU table" --to-hdf5

qiime tools import --input-path /mouse_pathabundance/mouse_pathabundance_unstratified.biom \
    --type 'FeatureTable[Frequency]' --input-format BIOMV210Format \
    --output-path mouse_pathabundance_unstratified.qza

qiime tools import --input-path mouse_genefamilies_ko_unstratified.biom \
    --type 'FeatureTable[Frequency]' --input-format BIOMV210Format \
    --output-path mouse_genefamilies_ko_unstratified.qza

qiime feature-table filter-samples \
    --i-table mouse_pathabundance_unstratified.qza \
    --m-metadata-file mapping_updated_072722.txt --p-where "[Age]='adult'" \
    --o-filtered-table mouse_pathabundance_unstratified_noinfant.qza

qiime feature-table filter-samples \
    --i-table mouse_genefamilies_ko_unstratified.qza \
    --m-metadata-file mapping_genefam_updated_072722.txt --p-where "[Age]='adult'" \
    --o-filtered-table mouse_genefamilies_ko_unstratified_noinfant.qza

qiime diversity core-metrics --i-table mouse_pathabundance_unstratified_noinfant.qza \
    --p-sampling-depth 4500000 --m-metadata-file mapping_update_072722_noinfant.txt \
    --output-dir pathabundance_unstratified_noinfant

qiime diversity core-metrics --i-table mouse_genefamilies_ko_unstratified_noinfant.qza \
    --p-sampling-depth 13000000 --m-metadata-file mapping_genefam_update_072722_noinfant.txt \
    --output-dir genefamilies_unstratified_noinfant


### Diversity metrics were exported from QIIME2 

Prior to downstream statistical analysis in R, we exported beta diversity distance matrices.

In [ ]:
qiime tools export --input-path /pathabundance_unstratified_noinfant/bray_curtis_distance_matrix.qza \
    --output-path /pathabundance_unstratified_noinfant/bray-curtis-distance-matrix

qiime tools export --input-path /pathabundance_unstratified_noinfant/jaccard_distance_matrix.qza \
    --output-path /pathabundance_unstratified_noinfant/jaccard-distance-matrix

qiime tools export --input-path /genefamilies_unstratified_noinfant/bray_curtis_distance_matrix.qza \
    --output-path /genefamilies_unstratified_noinfant/bray-curtis-distance-matrix

qiime tools export --input-path //genefamilies_unstratified_noinfant/jaccard_distance_matrix.qza \
    --output-path /genefamilies_unstratified_noinfant/jaccard-distance-matrix


We also exported alpha diversity indices.

In [ ]:
qiime tools export --input-path /genefamilies_unstratified/evenness_vector.qza \
    --output-path /genefamilies_unstratified/evenness

qiime tools export --input-path /genefamilies_unstratified/observed_otus_vector.qza \
    --output-path /genefamilies_unstratified/observed_otus

qiime tools export --input-path /genefamilies_unstratified/shannon_vector.qza \
    --output-path /Users/elizabethmallott/Dropbox/Projects/Gut_microbiome/mouse_inoculation/shotgun/humann2_output/genefamilies_unstratified/shannon

qiime tools export --input-path /pathabundance_unstratified/evenness_vector.qza \
    --output-path /pathabundance_unstratified/evenness

qiime tools export --input-path /pathabundance_unstratified/observed_otus_vector.qza \
    --output-path /pathabundance_unstratified/observed_otus

qiime tools export --input-path /pathabundance_unstratified/shannon_vector.qza \
    --output-path /pathabundance_unstratified/shannon
